# Track Token Usage

In this notebook I go over how to track token usage in LangChain. At the time of creating this document, token tracking is only implemented for OpenAI.

Let's first look at an extremely simple example of tracking token usage for a single LLM call. Before we begin, let's load in our environment variables.

In [4]:
from dotenv import load_dotenv
import os

%load_ext dotenv
%dotenv

In [5]:
openai_api_key  = os.environ['OPANAI_API_KEY']

In [6]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

In [8]:
llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2, openai_api_key=openai_api_key)

In [9]:
with get_openai_callback() as cb:
    print(cb)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [10]:
with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    print(cb)

Tokens Used: 42
	Prompt Tokens: 4
	Completion Tokens: 38
Successful Requests: 1
Total Cost (USD): $0.00084


Getting attributes of the callback return

In [11]:
with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    print(cb.total_cost)
    print(cb.total_tokens)

0.00084
42


### Tracking Cost When Using Chain

A chain consists of multiple LLM calls, we can track a total of all these costs using the `get_openai_callback` function. Here's an example of how exactly we can do that:

In [15]:
from langchain.agents import AgentType, load_tools, initialize_agent
from langchain.llms import OpenAI


llm = OpenAI(openai_api_key=openai_api_key, temperature=0)
tools = load_tools(["llm-math"], llm=llm)

agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

In [16]:
with get_openai_callback() as cb:
    agent.run("What is the product of 5 and 20")
    print(cb)

Tokens Used: 631
	Prompt Tokens: 572
	Completion Tokens: 59
Successful Requests: 3
Total Cost (USD): $0.012620000000000003


In [18]:
with get_openai_callback() as cb:
    response = agent.run("What is the product of 5 and 20")
    print(response)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

The product of 5 and 20 is 100.
Total Tokens: 631
Prompt Tokens: 572
Completion Tokens: 59
Total Cost (USD): $0.012620000000000003
